In [1]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
from matplotlib import pyplot as plt
import math
import datetime
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.text import Text
import string, re


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kriza\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kriza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kriza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kriza\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kriza\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

# SENTIMENT ANALYZER
### POS Legend
https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

In [3]:
def GetStopWords():
    from nltk.corpus import stopwords
    stop_words = list(set(stopwords.words('english')))
    return stop_words

def RemovePuncFromSentence(sentence):
    sentence = re.sub(r'[^\w\s]','', sentence)
    return sentence

def RemoveStopWords(stop_words, sentence):
    return [w for w in sentence if not w in stop_words]


def PrepareColumnForSentimentAnalysis(df, column):
    #go through col and remove punc and create new array for future new col
    newColName = "cleaned_"+column
    stop_words = GetStopWords()
    word_types = ["J","R","V", "N"]
    
    cleanedCol = []
    for index, row in df.iterrows():
        sentences = sent_tokenize(row[column])
        cleanSentences = [RemovePuncFromSentence(sentence) for sentence in sentences]
        
        #next step removing stop words
        words = [word_tokenize(sentence) for sentence in cleanSentences]
        filtered = [RemoveStopWords(stop_words, s) for s in words]
        posWords = [nltk.pos_tag(tokenized_sent) for tokenized_sent in filtered]
        cleanedSentence = []
        for sent in posWords:
            for w in sent:
                if w[1][0] in word_types:
                    cleanedSentence.append(w[0])
                    
        justString = " ".join(str(x) for x in cleanedSentence)            
        cleanedCol.append(justString)
            
    df[newColName] = cleanedCol
    return df

def GetSentimentScore(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    return(str(score))
    
def GetSentimentOfColumn(df, column):
    newCol = column
    Sentiment_neg = []
    Sentiment_pos = []
    Sentiment_neut = []
    Sentiment_comp = []

    for index, row in df.iterrows():
       
        stringresult = GetSentimentScore(row[column])
        stringresult = stringresult.replace('{', '').replace('}', '')
        
        Sentiment_neg.append(float((stringresult.split(',')[0]).split(':')[1]))
        Sentiment_neut.append(float((stringresult.split(',')[1]).split(':')[1]))
        Sentiment_pos.append(float((stringresult.split(',')[2]).split(':')[1]))
        Sentiment_comp.append(float((stringresult.split(',')[3]).split(':')[1]))
        
    
    df[newCol+"_sent_neg"] = Sentiment_neg  
    df[newCol+"_sent_neut"] = Sentiment_neut  
    df[newCol+"_sent_pos"] = Sentiment_pos  
    df[newCol+"_sent_comp"] = Sentiment_comp  
    return df
        